# TMDB API로 데이터 수집

## TMDB 드라마 크레딧 데이터 추가 수집

1. 범위: TMDB 드라마 크레딧(프로듀서/배우/작가) 데이터
2. 방식: 기존에 수집한 드라마 파일 속 각 프로듀서/배우/작가에 대한 세부 데이터 수집
3. 목적: 현재 데이터는 파싱이 필요한데, 각 컬럼별 값끼리 매칭을 확실히 하기 위해 데이터 수집 단계에서부터 파싱하여 컬럼을 분리하고자 함.
4. 조건: 누락된 페이지가 0%여야 함, TMDB API 초당 요청 수(40 req/s) 제한 대응, 500페이지 제한 대응, 중단/재개 지원, 수집 파일은 각 멀티값을 파싱하여 컬럼 분리하고, 프로듀서/배우/작가별 테이블 분리하여 3개의 Parquet로 저장, Jupyter에서 그대로 실행 가능한 코드
5. 필수 컬럼 (23개):
- executive_producer: crew_member_id, name, gender, popularity, department, job, series_id, profile_path
- writer: crew_member_id, name, gender, popularity, department, job, series_id, profile_path
- cast: cast_id, name, gender, popularity, character, series_id, profile_path

6. 수집 시 주의사항
- crew에는 프로듀서/작가 외 다양한 직군(department/job)이 포함되므로, 필터 정확히 해야 함
- 감독: crew 안에서 get("job") == "Executive Producer", 공동 감독 있을 수 있음 (대소문자 구분)
- 작가: 원안, 각본, 집필자 모두 커버하려면 다음과 같은 조건 필요, 공동 작가 있을 수 있음 (대소문자 구분)
writers = [c for c in crew_list if c.get("job") in ("Writer", "Screenplay", "Story", "Creator", "Novel", "Comic Book") or c.get("department") == "Writing"]
- 주연 배우 top 5: cast 배열을 order 필드를 기준으로 오름차순 정렬 (반드시 주연 배우부터 5명 이름으로 수집, 절대 popularity 기준 쓰면 안 됨)
filtered = [c for c in sorted_cast if c.get("character")]
top_5 = filtered[:5]

7. 공식문서: https://developer.themoviedb.org/reference/tv-series-credits
8. 수집 방식: 멀티 스레드 방식
9. 수집 결과: 총 3,581개 수집 완료, 약 1.5분 소요

## 영화 크레딧 수집 코드 (파싱 후 정규화)

In [12]:
# ==========================================================
# TMDB DRAMA CREDITS COLLECTOR V3 (TV Series, Producer/Writer/Cast)
# - 기준: 기존 영화 크레딧 수집 코드 스타일 (멀티스레드 + requests)
# - 대상: 미리 필터된 "드라마 시리즈" 파일 속 TMDB TV id 목록
# - 기능:
#   * /tv/{series_id}/credits 기반 producer / writer / top 5 cast "행 분리" 수집
#   * 초당 40req/s 이하 Rate Limiter (thread-safe)
#   * 중단/재개 지원 (checkpoint.json + 임시 CSV 누적)
#   * 최종 출력: 3개의 Parquet
#       - tv_dramas_producer.parquet
#       - tv_dramas_writer.parquet
#       - tv_dramas_cast.parquet
# ==========================================================

import os
import json
import time
import threading
from collections import deque
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from dotenv import load_dotenv
from tqdm import tqdm

# ==========================================================
# 설정
# ==========================================================

# (1) 입력: 드라마 시리즈 파일 & TMDB id 컬럼명
INPUT_PARQUET = "dramas_merged_final.parquet"
ID_COLUMN = "id"   # TMDB TV series id 컬럼명

# (2) 출력 디렉토리
OUTPUT_DIR = Path("files")
OUTPUT_DIR.mkdir(exist_ok=True)

# (3) 임시 CSV & 최종 Parquet 경로
PRODUCER_TEMP_CSV = OUTPUT_DIR / "tv_dramas_producer_temp.csv"
WRITER_TEMP_CSV   = OUTPUT_DIR / "tv_dramas_writer_temp.csv"
CAST_TEMP_CSV     = OUTPUT_DIR / "tv_dramas_cast_temp.csv"

PRODUCER_PARQUET  = OUTPUT_DIR / "tv_dramas_producer.parquet"
WRITER_PARQUET    = OUTPUT_DIR / "tv_dramas_writer.parquet"
CAST_PARQUET      = OUTPUT_DIR / "tv_dramas_cast.parquet"

# (4) 체크포인트 & 실패 ID 기록
CHECKPOINT_PATH   = OUTPUT_DIR / "tv_dramas_credits_checkpoint.json"
FAILED_IDS_PATH   = OUTPUT_DIR / "tv_dramas_credits_failed_ids.json"

# (5) TMDB API
HEADERS = {"accept": "application/json"}
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise ValueError("TMDB_API_KEY 가 .env 에 설정되어 있지 않습니다.")

# (6) Rate Limit & Thread 설정
MAX_CALLS_PER_SECOND = 40   # TMDB 문서 기준 여유 있게 사용
MAX_WORKERS = 30            # ThreadPoolExecutor worker 수
FLUSH_EVERY = 500           # N개 시리즈 처리마다 임시 CSV + 체크포인트 flush
MAX_RETRIES = 3             # 개별 요청 재시도 횟수

WRITER_JOBS = {
    "Writer", "Screenplay", "Story", "Creator",
    "Novel", "Comic Book", "Adaptation", "Screenstory"
}
WRITER_JOBS_LOWER = {j.lower() for j in WRITER_JOBS}

# ==========================================================
# Rate Limiter (thread-safe)
# ==========================================================

class RateLimiter:
    """초당 max_calls 이하로 요청을 제한하는 Rate Limiter (thread-safe)"""
    def __init__(self, max_calls: int, period: float = 1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock = threading.Lock()

    def acquire(self):
        with self.lock:
            now = time.time()

            # 기간 지난 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            # 초과 시 대기
            if len(self.calls) >= self.max_calls:
                sleep_for = self.period - (now - self.calls[0]) + 0.01
                if sleep_for > 0:
                    time.sleep(sleep_for)

                now = time.time()
                while self.calls and now - self.calls[0] > self.period:
                    self.calls.popleft()

            self.calls.append(now)

rate_limiter = RateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# HTTP Session (thread-local)
# ==========================================================

thread_local = threading.local()

def create_session() -> requests.Session:
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504],
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50,
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def get_session() -> requests.Session:
    """각 쓰레드별로 독립적인 Session 사용"""
    if not hasattr(thread_local, "session"):
        thread_local.session = create_session()
    return thread_local.session

# ==========================================================
# 체크포인트 유틸
# ==========================================================

def load_checkpoint():
    """이미 처리한 시리즈 ID 정보 로드"""
    if CHECKPOINT_PATH.exists():
        try:
            data = json.loads(CHECKPOINT_PATH.read_text(encoding="utf-8"))
            done_ids = set(data.get("done_ids", []))
            failed_ids = set(data.get("failed_ids", []))
            return done_ids, failed_ids
        except Exception:
            pass
    return set(), set()

def save_checkpoint(done_ids, failed_ids):
    """체크포인트 저장"""
    data = {
        "done_ids": sorted(list(done_ids)),
        "failed_ids": sorted(list(failed_ids)),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    tmp = CHECKPOINT_PATH.with_suffix(".tmp")
    tmp.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
    tmp.replace(CHECKPOINT_PATH)

# ==========================================================
# 임시 CSV append 유틸 (단일 스레드에서만 호출)
# ==========================================================

def append_temp_csv(df: pd.DataFrame, path: Path):
    """
    임시 CSV에 DataFrame을 append (header는 처음 한 번만)
    - main thread에서만 호출 → race condition 없음
    """
    if df is None or df.empty:
        return
    header = not path.exists()
    df.to_csv(path, mode="a", header=header, index=False, encoding="utf-8")

# ==========================================================
# TV Series Credits Fetcher (핵심)
# ==========================================================

def get_tv_credit(series_id: int):
    """
    TV 시리즈의 cast / writer / producer 크레딧 수집 (행 단위)
    - Producer: job == "Executive Producer" (소문자 비교)
    - Writer: job ∈ WRITER_JOBS (대소문자 무시) 또는 department == "Writing"
    - Cast: order 기준 정렬 후, character가 존재하고 비어있지 않은 배우만 필터 → 그 중 Top 5
    """
    session = get_session()
    url = f"https://api.themoviedb.org/3/tv/{series_id}/credits"
    params = {"api_key": API_KEY, "language": "en-US"}

    data = None
    for attempt in range(MAX_RETRIES):
        try:
            rate_limiter.acquire()
            resp = session.get(url, params=params, headers=HEADERS, timeout=10)

            # 404: 크레딧 없음 → 그냥 None
            if resp.status_code == 404:
                return None

            # 429: rate limit → Retry-After 또는 지수 backoff
            if resp.status_code == 429:
                retry_after = resp.headers.get("Retry-After", None)
                try:
                    sleep_for = float(retry_after) if retry_after is not None else 2 ** attempt
                except (ValueError, TypeError):
                    sleep_for = 2 ** attempt
                print(f"[429] series_id={series_id}, attempt={attempt+1}, sleep={sleep_for}s")
                time.sleep(sleep_for)
                continue

            resp.raise_for_status()
            data = resp.json()
            break  # 성공 → 루프 탈출

        except requests.exceptions.RequestException as e:
            # 마지막 재시도에서도 실패하면 포기
            if attempt == MAX_RETRIES - 1:
                print(f"[ERROR] series_id={series_id} 최종 실패: {e}")
                return None
            sleep_for = 2 ** attempt
            print(f"[WARN] series_id={series_id}, attempt={attempt+1}, error={e}, retry in {sleep_for}s")
            time.sleep(sleep_for)

    # 모든 시도 실패
    if data is None:
        return None

    cast = data.get("cast", []) or []
    crew = data.get("crew", []) or []

    # =============================
    # 1) CAST 처리 (Top 5 by order, character 필터)
    # =============================
    if cast:
        cast_df = pd.DataFrame(cast)

        # order 기준 정렬
        if "order" in cast_df.columns:
            cast_df = cast_df.sort_values(by="order")

        # 🔴 요구사항: character가 존재하고 비어있지 않은 배우만
        if "character" in cast_df.columns:
            cast_df = cast_df[
                cast_df["character"].notna()
                & (cast_df["character"].astype(str).str.strip() != "")
            ]
        else:
            # character 컬럼 자체가 없으면 주연 정의가 불가능하므로 비워둠
            cast_df = cast_df.iloc[0:0]

        # 필터링 후 상위 5명
        cast_df = cast_df.head(5)

        # 필요한 컬럼만 선택
        cast_cols_select = ["id", "name", "gender", "popularity",
                            "profile_path", "character"]
        cast_existing = [c for c in cast_cols_select if c in cast_df.columns]
        cast_df = cast_df[cast_existing].copy()
        cast_df = cast_df.rename(columns={"id": "cast_id"})

        # character 컬럼 보정
        if "character" not in cast_df.columns:
            cast_df["character"] = ""
        else:
            cast_df["character"] = cast_df["character"].fillna("").astype(str)

        cast_df["series_id"] = int(series_id)
    else:
        cast_df = pd.DataFrame()

    # =============================
    # 2) CREW → WRITER / PRODUCER
    # =============================
    if crew:
        crew_df = pd.DataFrame(crew)

        crew_cols_select = [
            "id", "name", "gender", "popularity",
            "profile_path", "department", "job"
        ]
        existing = [c for c in crew_cols_select if c in crew_df.columns]
        crew_df = crew_df[existing].copy()
        crew_df = crew_df.rename(columns={"id": "crew_member_id"})
        crew_df["series_id"] = int(series_id)

        # 2-1) PRODUCER (Executive Producer만, 소문자 비교)
        if "job" in crew_df.columns:
            producer_df = crew_df[
                crew_df["job"].fillna("").str.lower() == "executive producer"
            ].copy()
        else:
            producer_df = pd.DataFrame()

        # 2-2) WRITER (job: WRITER_JOBS_LOWER, department: writing)
        if "job" in crew_df.columns and "department" in crew_df.columns:
            job_lower = crew_df["job"].fillna("").str.lower()
            dept_lower = crew_df["department"].fillna("").str.lower()
            writer_mask = job_lower.isin(WRITER_JOBS_LOWER) | (dept_lower == "writing")
            writer_df = crew_df[writer_mask].copy()
        elif "job" in crew_df.columns:
            job_lower = crew_df["job"].fillna("").str.lower()
            writer_mask = job_lower.isin(WRITER_JOBS_LOWER)
            writer_df = crew_df[writer_mask].copy()
        else:
            writer_df = pd.DataFrame()
    else:
        producer_df = pd.DataFrame()
        writer_df = pd.DataFrame()

    # =============================
    # 3) 컬럼 순서 정렬 + 누락 컬럼 채우기
    # =============================
    producer_cols = [
        "crew_member_id", "name", "gender", "popularity",
        "department", "job", "series_id", "profile_path"
    ]
    writer_cols   = [
        "crew_member_id", "name", "gender", "popularity",
        "department", "job", "series_id", "profile_path"
    ]
    cast_cols     = [
        "cast_id", "name", "gender", "popularity",
        "character", "series_id", "profile_path"
    ]

    # Producer
    for col in producer_cols:
        if col not in producer_df.columns:
            producer_df[col] = pd.NA
    producer_df = producer_df[producer_cols]

    # Writer
    for col in writer_cols:
        if col not in writer_df.columns:
            writer_df[col] = pd.NA
    writer_df = writer_df[writer_cols]

    # Cast
    for col in cast_cols:
        if col not in cast_df.columns:
            cast_df[col] = pd.NA
    cast_df = cast_df[cast_cols]

    return {
        "producer": producer_df,
        "writer":   writer_df,
        "cast":     cast_df,
    }

# ==========================================================
# 메인 파이프라인
# ==========================================================

def main():
    start_time = time.time()
    print("=" * 80)
    print("🚀 TMDB DRAMA CREDITS COLLECTOR V3 (TV Series)")
    print("=" * 80)

    # ----- 입력 파일 로드 -----
    ipath = Path(INPUT_PARQUET)
    if not ipath.exists():
        raise FileNotFoundError(f"입력 파일을 찾을 수 없습니다: {ipath.resolve()}")

    dramas = pd.read_parquet(ipath)
    if ID_COLUMN not in dramas.columns:
        raise KeyError(f"입력 파일에 '{ID_COLUMN}' 컬럼이 없습니다.")

    ids = (
        dramas[ID_COLUMN]
        .dropna()
        .astype(int)
        .unique()
        .tolist()
    )
    ids = sorted(ids)
    print(f"📂 드라마 시리즈 수: {len(ids):,}개 (unique TMDB TV id)")

    # ----- 체크포인트 로드 -----
    done_ids, failed_ids = load_checkpoint()
    print(f"✅ 이미 완료된 시리즈: {len(done_ids):,}개")
    print(f"⚠️ 이전 실패 시리즈: {len(failed_ids):,}개")

    remaining_ids = [i for i in ids if i not in done_ids and i not in failed_ids]
    print(f"⏳ 이번에 처리할 시리즈: {len(remaining_ids):,}개")

    # ----- 크레딧 수집 -----
    producer_buf = []
    writer_buf = []
    cast_buf = []
    processed = 0

    if remaining_ids:
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = {
                executor.submit(get_tv_credit, series_id): series_id
                for series_id in remaining_ids
            }

            for future in tqdm(
                as_completed(futures),
                total=len(futures),
                desc="드라마 크레딧 수집",
                unit="series",
            ):
                series_id = futures[future]
                try:
                    result = future.result()
                except Exception as e:
                    print(f"[EXCEPTION] 시리즈 ID {series_id} 처리 중 예외: {e}")
                    failed_ids.add(series_id)
                    continue

                if result is None:
                    failed_ids.add(series_id)
                else:
                    if not result["producer"].empty:
                        producer_buf.append(result["producer"])
                    if not result["writer"].empty:
                        writer_buf.append(result["writer"])
                    if not result["cast"].empty:
                        cast_buf.append(result["cast"])
                    done_ids.add(series_id)

                processed += 1

                # 주기적으로 flush + checkpoint
                if processed % FLUSH_EVERY == 0:
                    if producer_buf:
                        append_temp_csv(
                            pd.concat(producer_buf, ignore_index=True),
                            PRODUCER_TEMP_CSV,
                        )
                        producer_buf = []
                    if writer_buf:
                        append_temp_csv(
                            pd.concat(writer_buf, ignore_index=True),
                            WRITER_TEMP_CSV,
                        )
                        writer_buf = []
                    if cast_buf:
                        append_temp_csv(
                            pd.concat(cast_buf, ignore_index=True),
                            CAST_TEMP_CSV,
                        )
                        cast_buf = []

                    save_checkpoint(done_ids, failed_ids)
                    print(f"\n💾 체크포인트 저장 (processed={processed:,})")

        # 마지막 남은 버퍼 flush
        if producer_buf:
            append_temp_csv(pd.concat(producer_buf, ignore_index=True), PRODUCER_TEMP_CSV)
        if writer_buf:
            append_temp_csv(pd.concat(writer_buf, ignore_index=True), WRITER_TEMP_CSV)
        if cast_buf:
            append_temp_csv(pd.concat(cast_buf, ignore_index=True), CAST_TEMP_CSV)

        save_checkpoint(done_ids, failed_ids)
        FAILED_IDS_PATH.write_text(
            json.dumps(sorted(list(failed_ids)), ensure_ascii=False, indent=2),
            encoding="utf-8",
        )
        print("\n✅ 크레딧 수집 단계 완료")
    else:
        print("\n모든 시리즈가 이미 처리되었습니다. 바로 Parquet 변환으로 넘어갑니다.")

    # ======================================================
    # 임시 CSV → Parquet 변환
    # ======================================================
    print("\n📦 임시 CSV → Parquet 변환")

    # Producer
    if PRODUCER_TEMP_CSV.exists():
        df_prod = pd.read_csv(PRODUCER_TEMP_CSV)
        prod_cols = [
            "crew_member_id", "name", "gender", "popularity",
            "department", "job", "series_id", "profile_path",
        ]
        for c in prod_cols:
            if c not in df_prod.columns:
                df_prod[c] = pd.NA
        df_prod = df_prod[prod_cols]
        df_prod.to_parquet(PRODUCER_PARQUET, index=False)
        print(f"  ▶ Producer Parquet 저장: {PRODUCER_PARQUET} ({len(df_prod):,} rows)")
    else:
        print("  ⚠️ Producer 임시 CSV가 없습니다 (데이터 없음).")

    # Writer
    if WRITER_TEMP_CSV.exists():
        df_writer = pd.read_csv(WRITER_TEMP_CSV)
        writer_cols = [
            "crew_member_id", "name", "gender", "popularity",
            "department", "job", "series_id", "profile_path",
        ]
        for c in writer_cols:
            if c not in df_writer.columns:
                df_writer[c] = pd.NA
        df_writer = df_writer[writer_cols]
        df_writer.to_parquet(WRITER_PARQUET, index=False)
        print(f"  ▶ Writer Parquet 저장: {WRITER_PARQUET} ({len(df_writer):,} rows)")
    else:
        print("  ⚠️ Writer 임시 CSV가 없습니다 (데이터 없음).")

    # Cast
    if CAST_TEMP_CSV.exists():
        df_cast = pd.read_csv(CAST_TEMP_CSV)
        cast_cols = [
            "cast_id", "name", "gender", "popularity",
            "character", "series_id", "profile_path",
        ]
        for c in cast_cols:
            if c not in df_cast.columns:
                df_cast[c] = pd.NA
        df_cast = df_cast[cast_cols]
        df_cast.to_parquet(CAST_PARQUET, index=False)
        print(f"  ▶ Cast Parquet 저장: {CAST_PARQUET} ({len(df_cast):,} rows)")
    else:
        print("  ⚠️ Cast 임시 CSV가 없습니다 (데이터 없음).")

    elapsed = time.time() - start_time
    print("\n" + "=" * 80)
    print("🎉 TMDB DRAMA CREDITS 수집 파이프라인 완료 (V3)")
    print(f"⏱ 총 소요시간: {elapsed/60:.1f}분 ({elapsed/3600:.2f}시간)")
    print(f"✅ 완료 시리즈: {len(done_ids):,}개")
    print(f"⚠️ 실패 시리즈: {len(failed_ids):,}개 (파일: {FAILED_IDS_PATH})")
    print("=" * 80)


if __name__ == "__main__":
    main()


🚀 TMDB DRAMA CREDITS COLLECTOR V3 (TV Series)
📂 드라마 시리즈 수: 3,581개 (unique TMDB TV id)
✅ 이미 완료된 시리즈: 0개
⚠️ 이전 실패 시리즈: 0개
⏳ 이번에 처리할 시리즈: 3,581개


드라마 크레딧 수집:  14%|█▎        | 486/3581 [00:12<01:38, 31.39series/s]


💾 체크포인트 저장 (processed=500)


드라마 크레딧 수집:  28%|██▊       | 999/3581 [00:25<01:00, 42.34series/s]


💾 체크포인트 저장 (processed=1,000)


...


💾 체크포인트 저장 (processed=3,000)


드라마 크레딧 수집:  98%|█████████▊| 3510/3581 [01:28<00:01, 49.87series/s]


💾 체크포인트 저장 (processed=3,500)


드라마 크레딧 수집: 100%|██████████| 3581/3581 [01:30<00:00, 39.49series/s]


✅ 크레딧 수집 단계 완료

📦 임시 CSV → Parquet 변환
  ▶ Producer Parquet 저장: files/tv_dramas_producer.parquet (7,871 rows)
  ▶ Writer Parquet 저장: files/tv_dramas_writer.parquet (3,676 rows)
  ▶ Cast Parquet 저장: files/tv_dramas_cast.parquet (16,692 rows)

🎉 TMDB DRAMA CREDITS 수집 파이프라인 완료 (V3)
⏱ 총 소요시간: 1.5분 (0.03시간)
✅ 완료 시리즈: 3,581개
⚠️ 실패 시리즈: 0개 (파일: files/tv_dramas_credits_failed_ids.json)


In [13]:
tv_dramas_cast = pd.read_parquet('tv_dramas_cast.parquet')
tv_dramas_writer = pd.read_parquet('tv_dramas_writer.parquet')
tv_dramas_producer = pd.read_parquet('tv_dramas_producer.parquet')

In [14]:
tv_dramas_cast

,cast_id,name,gender,popularity,character,series_id,profile_path
0,12795,Nikolaj Coster-Waldau,2,2.9643,John Amsterdam,1144,/6w2SgB20qzs2R5MQIAckINLhfoP.jpg
1,20803,Zuleikha Robinson,1,1.4830,Eva Marquez,1144,/838l1DBT6LqxI9Lw5qXrDpIqEzW.jpg
2,196179,Stephen McKinley Henderson,2,1.1218,Omar,1144,/z2weSPo4sdMNj47tP5o0me41r2z.jpg
3,87669,Alexie Gilmore,1,0.7575,Dr. Sara Dillane,1144,/1Ev9eTuu8bL6t8xzfqchZm4wSfI.jpg
4,2005045,Jian Leonardo,2,0.3031,Detective,1144,None
...,...,...,...,...,...,...,...
16687,1299479,Lee Jin-uk,2,3.3383,Yoon Suk-hoon,281004,/fODHfrMuRac7xkeFErJOjawggM1.jpg
16688,1813385,Jung Chae-yeon,1,2.2840,Kang Hyo-min / Kang Hyo-Ju,281004,/xdE2sxSVRf0JMLoyqTpwY3oU5Bl.jpg
16689,1115730,Jeon Hye-bin,1,1.4287,Hyo Min-jung,281004,/Ar9Ol7gk1YQdQqdNaYEgzcIEstY.jpg
16690,1615316,Lee Hak-ju,2,1.0675,Lee Jin-woo,281004,/gajjcFdyWx49Mlca4JwTVe3zr87.jpg


In [15]:
tv_dramas_cast.isna().mean().sort_values(ascending=False)

profile_path    0.030374
cast_id         0.000000
name            0.000000
gender          0.000000
popularity      0.000000
character       0.000000
series_id       0.000000
dtype: float64

In [16]:
tv_dramas_writer

,crew_member_id,name,gender,popularity,department,job,series_id,profile_path
0,1318326,Jacob Cooney,2,0.3665,Writing,Writers' Assistant,36,/4hEMgTX1BBXXNPGNFga8xY3e7MS.jpg
1,1223164,Cecily von Ziegesar,0,0.0654,Writing,Novel,1395,None
2,7624,Stan Lee,2,2.7493,Writing,Comic Book,1403,/kKeyWoFtTqOPsbmwylNHmuB3En9.jpg
3,18866,Jack Kirby,2,1.0420,Writing,Comic Book,1403,/nSD0Q3gt7SQpsqyScRoUp1Hjh60.jpg
4,1855651,Iván Cuevas,2,0.0963,Writing,Writer,1341,None
...,...,...,...,...,...,...,...,...
3671,1302761,Toni Carrizosa,0,0.2087,Writing,Idea,280888,/gfcWPejP672bt0ODKmWbgZQ8kXJ.jpg
3672,4102600,Verónica Vila San Juan,0,0.0966,Writing,Idea,280888,None
3673,5638231,Dario Venegas,0,0.0071,Writing,Writer,297464,None
3674,1563037,Lina Uribe,1,0.1445,Writing,Writer,297464,None


In [17]:
tv_dramas_writer.isna().mean().sort_values(ascending=False)

profile_path      0.597116
crew_member_id    0.000000
name              0.000000
gender            0.000000
popularity        0.000000
department        0.000000
job               0.000000
series_id         0.000000
dtype: float64

In [18]:
tv_dramas_producer

,crew_member_id,name,gender,popularity,department,job,series_id,profile_path
0,54773,Steven Pearl,2,0.2786,Production,Executive Producer,1144,None
1,68901,David Manson,2,0.3145,Production,Executive Producer,1144,/4EYuvfQ9Z5tAyQ7lT6xfKzUietR.jpg
2,34050,Allan Loeb,2,0.4953,Production,Executive Producer,1144,/xtDHFBb0QQ3QGvdPSextyouOBQr.jpg
3,168812,Stephanie Savage,1,0.2975,Production,Executive Producer,1395,/bWCBO6nDtYsUrPXZiFsqrMvtZgN.jpg
4,2609135,Mark A. Burley,2,0.0967,Production,Executive Producer,186,None
...,...,...,...,...,...,...,...,...
7866,3723056,Ana Eiras,1,0.1719,Production,Executive Producer,280888,/jCw8MIggxVdXNNshz4U6NhPLVlx.jpg
7867,1311136,Jang Kyung-ik,2,0.8182,Production,Executive Producer,280945,None
7868,3984952,Walter Iuzzolino,2,0.0867,Production,Executive Producer,279603,None
7869,3358016,Jo McGrath,0,0.1432,Production,Executive Producer,279603,None


In [19]:
tv_dramas_producer.isna().mean().sort_values(ascending=False)

profile_path      0.440097
crew_member_id    0.000000
name              0.000000
gender            0.000000
popularity        0.000000
department        0.000000
job               0.000000
series_id         0.000000
dtype: float64

## 기존 영화 크레딧 수집 코드 (파싱 전)

In [1]:
# ==========================================================
# TMDB MOVIE FULL COLLECTOR — PRODUCTION V9 (Extreme-Day FULL)
# (2005-01-01 ~ 2025-11-30)
#
# - 대상: TMDB 영화 (vote_count >= 30 + imdb_id 필수)
# - 특징:
#   * discover/movie 기간 자동 분할 (500페이지 제한 대응)
#   * ✅ 어떤 구간도 500페이지 초과 상태로 수집하지 않음
#   * ✅ 만약 "단일 날짜"에서 total_pages > 500 발생 시 → RuntimeError 발생
#       → TMDB API 한계로 그 날짜는 전수 수집 불가하다는 것을 명시적으로 알려줌
#       → 즉, "조용한 누락"은 절대 없음 (0% 누락 보장 or 실패)
#   * 감독 / 작가 / 주연배우(Top 5) 크레딧 전수 수집
#   * 초당 요청 제한(35 req/s) + 재시도(backoff)
#   * JSONL 스트림 저장 + Parquet 최종 저장
#   * 중단/재개: checkpoint + JSONL + failed_ids
# ==========================================================

import os
import json
import asyncio
import aiohttp
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import re
from dotenv import load_dotenv
load_dotenv()

try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception:
    pass

# ==========================================================
# CONFIG
# ==========================================================
DATA_DIR = Path("./tmdb_movie_full")
DATA_DIR.mkdir(exist_ok=True)

MOVIE_IDS_PATH     = DATA_DIR / "movie_ids_2005_2025.json"
DETAILS_JSONL_PATH = DATA_DIR / "movie_details_2005_2025.jsonl"
CHECKPOINT_PATH    = DATA_DIR / "checkpoint.json"
FAILED_IDS_PATH    = DATA_DIR / "failed_ids.json"
FINAL_PARQUET_PATH = DATA_DIR / "tmdb_movies_2005_2025_FULL.parquet"

API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise ValueError("환경변수 TMDB_API_KEY 설정 필요!")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS  = {"accept": "application/json"}

TIMEOUT   = 10
MAX_RETRIES = 5
MAX_CALLS_PER_SECOND   = 35
MAX_CONCURRENT_REQUESTS = 150  # 효율성

START_DATE = datetime(2005, 1, 1)
END_DATE   = datetime(2025, 11, 30)

WRITER_JOBS = {
    "Writer", "Screenplay", "Story", "Creator",
    "Novel", "Comic Book", "Adaptation", "Screenstory"
}

jsonl_lock      = asyncio.Lock()
checkpoint_lock = asyncio.Lock()

# ==========================================================
# Rate Limiter
# ==========================================================
class RateLimiter:
    def __init__(self, max_calls=35, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock  = asyncio.Lock()

    async def acquire(self):
        async with self.lock:
            loop = asyncio.get_running_loop()
            now = loop.time()

            # 오래된 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            if len(self.calls) >= self.max_calls:
                wait = self.period - (now - self.calls[0]) + 0.02
                if wait > 0:
                    await asyncio.sleep(wait)

                now = loop.time()
                while self.calls and now - self.calls[0] > self.period:
                    self.calls.popleft()

            self.calls.append(loop.time())


rate_limiter = RateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# fetch_json
# ==========================================================
async def fetch_json(session, url, params, desc="", retries=MAX_RETRIES):
    for attempt in range(1, retries + 1):
        await rate_limiter.acquire()

        try:
            async with session.get(url, params=params, timeout=TIMEOUT) as resp:
                if resp.status == 200:
                    return await resp.json()

                if resp.status == 404:
                    return None

                if resp.status in {429, 500, 502, 503, 504}:
                    if resp.status == 429:
                        retry_after = resp.headers.get("Retry-After", "2")
                        try:
                            retry_after = int(retry_after)
                        except Exception:
                            retry_after = 2
                        await asyncio.sleep(retry_after)
                    else:
                        await asyncio.sleep(attempt * 2)
                    continue

                # 기타 에러는 실패로 처리
                return None

        except asyncio.TimeoutError:
            await asyncio.sleep(attempt * 2)
        except Exception:
            await asyncio.sleep(attempt * 2)

    return None

# ==========================================================
# 문자열 정제 유틸
# ==========================================================
def sanitize_text_mild(text):
    """
    경미한 정제 (이름, 제목용)
    - 줄바꿈, HTML 태그, 연속 공백만 제거
    - 유니코드 보존 (한글, 일본어, 유럽어 등)
    """
    if not text:
        return ""
    
    text = str(text)
    
    # HTML 태그 제거
    text = re.sub(r"<[^>]+>", " ", text)
    # 줄바꿈, 탭 제거
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()


def sanitize_text_strong(text):
    """
    강력한 정제 (character용 - Parquet 안정성 우선)
    - 제어 문자, 특수 유니코드 제거
    - 일반 유니코드는 보존
    """
    if not text:
        return ""
    
    text = str(text)
    
    # HTML 태그 제거
    text = re.sub(r"<[^>]+>", " ", text)
    # HTML 엔티티 제거
    text = re.sub(r"&[a-zA-Z]+;", " ", text)
    # 제어 문자 제거
    text = re.sub(r"[\x00-\x1F\x7F-\x9F]", " ", text)
    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()


def join_values(arr, use_strong_sanitize=False):
    """
    리스트를 세미콜론으로 join
    - use_strong_sanitize=True: character용 (강력한 정제)
    - use_strong_sanitize=False: 이름/제목용 (경미한 정제)
    """
    if not arr:
        return ""
    
    sanitize_func = sanitize_text_strong if use_strong_sanitize else sanitize_text_mild
    cleaned = [sanitize_func(x) if x is not None else "" for x in arr]
    return "; ".join(cleaned)

# ==========================================================
# DISCOVER
# ==========================================================
async def discover_total_pages(session, start_date: datetime, end_date: datetime):
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "primary_release_date.asc",
        "primary_release_date.gte": start_date.strftime("%Y-%m-%d"),
        "primary_release_date.lte": end_date.strftime("%Y-%m-%d"),
        "vote_count.gte": 30,
        "include_adult": "false",
        "page": 1,
    }
    url = f"{BASE_URL}/discover/movie"
    data = await fetch_json(session, url, params)
    if not data:
        return 0
    return int(data.get("total_pages", 0))


async def split_ranges(session, start_dt: datetime, end_dt: datetime, limit: int = 500):
    """
    날짜 구간을 여러 개로 나누어
    각 구간의 discover total_pages가 limit 이하가 되도록 분할.

    - 만약 "단일 날짜(s == e)"에서조차 total_pages > limit가 나오면
      TMDB API 한계로 인해 그 날짜는 전수 수집 불가능하다는 의미이므로
      RuntimeError를 발생시켜 조용한 누락을 막음.
    """
    queue = deque([(start_dt, end_dt)])
    out = []

    while queue:
        s, e = queue.popleft()
        if s > e:
            continue

        total_pages = await discover_total_pages(session, s, e)
        if total_pages == 0:
            continue

        if total_pages <= limit:
            out.append((s, e, total_pages))
            continue

        days = (e - s).days

        # 단일 날짜인데도 500페이지 초과 → TMDB discover 한계로 전수 수집 불가
        if days <= 0:
            msg = (
                f"❌ TMDB discover 한계 초과: {s.date()} 의 total_pages={total_pages} > {limit}\n"
                f"   이 날짜에 대해 discover/movie API로는 500페이지(10,000편) 이상을 전수 수집할 수 없습니다.\n"
                f"   (primary_release_date는 날짜 단위만 지원, 시간 단위 분할 불가)\n"
                f"   → 필터 조건을 완화/변경하거나, 이 날짜를 분석 범위에서 제외해야 합니다."
            )
            raise RuntimeError(msg)

        # 날짜 구간을 반씩 나누어 total_pages가 500 이하가 되도록 재귀 분할
        mid_days = days // 2
        mid = s + timedelta(days=mid_days)

        # 왼쪽: s ~ mid
        queue.append((s, mid))
        # 오른쪽: mid+1 ~ e
        if mid < e:
            queue.append((mid + timedelta(days=1), e))

    return sorted(out, key=lambda x: x[0])


async def discover_movie_ids(session):
    print("📌 Discover 단계 시작 (movie_id 수집)")

    ranges = await split_ranges(session, START_DATE, END_DATE, limit=500)
    print(f"📅 분할된 날짜 구간: {len(ranges)}")

    all_ids = set()
    url = f"{BASE_URL}/discover/movie"

    for idx, (s, e, pages) in enumerate(ranges, start=1):
        print(f"  ▷ [{idx}/{len(ranges)}] {s.date()} ~ {e.date()} (pages={pages})")

        for p in range(1, pages + 1):
            params = {
                "api_key": API_KEY,
                "language": "en-US",
                "sort_by": "primary_release_date.asc",
                "primary_release_date.gte": s.strftime("%Y-%m-%d"),
                "primary_release_date.lte": e.strftime("%Y-%m-%d"),
                "vote_count.gte": 30,
                "include_adult": "false",
                "page": p,
            }
            data = await fetch_json(session, url, params)
            if not data:
                continue

            for item in data.get("results", []):
                if item.get("vote_count", 0) >= 30:
                    all_ids.add(item["id"])

    ids = sorted(all_ids)
    MOVIE_IDS_PATH.write_text(json.dumps(ids))
    print(f"🎉 Discover 완료: 고유 movie_id {len(ids):,}개 확보")

    return ids

# ==========================================================
# EXTRACTORS
# ==========================================================
def extract_directors(crew):
    if not crew:
        return "", "", "", ""

    seen = {}
    for c in crew:
        if c.get("job") == "Director":
            pid = c.get("id")
            if pid and pid not in seen:
                seen[pid] = c

    if not seen:
        return "", "", "", ""

    arr = sorted(seen.values(), key=lambda x: x["id"])

    names    = join_values([a.get("name") for a in arr], use_strong_sanitize=False)
    ids      = join_values([a.get("id") for a in arr], use_strong_sanitize=False)
    genders  = join_values([a.get("gender") for a in arr], use_strong_sanitize=False)
    profiles = join_values([a.get("profile_path") for a in arr], use_strong_sanitize=False)

    return names, ids, genders, profiles


def extract_writers(crew):
    """
    작가 추출:
    - job이 WRITER_JOBS에 있는 경우
    - 또는 department == "Writing" 이면서 job이 존재하는 경우
    """
    if not crew:
        return "", "", "", "", ""

    seen = {}

    for c in crew:
        job  = c.get("job")
        dept = c.get("department")
        pid  = c.get("id")

        if not pid:
            continue

        if job in WRITER_JOBS:
            if pid not in seen:
                seen[pid] = {"person": c, "jobs": set()}
            seen[pid]["jobs"].add(job)
        elif dept == "Writing" and job:
            if pid not in seen:
                seen[pid] = {"person": c, "jobs": set()}
            seen[pid]["jobs"].add(job)

    if not seen:
        return "", "", "", "", ""

    arr = sorted(seen.items(), key=lambda x: x[0])

    names    = join_values([v["person"].get("name") for _, v in arr], use_strong_sanitize=False)
    ids      = join_values([v["person"].get("id") for _, v in arr], use_strong_sanitize=False)
    roles    = join_values([
        "/".join(sorted(v["jobs"])) if v["jobs"] else "Writer"
        for _, v in arr
    ], use_strong_sanitize=False)
    genders  = join_values([v["person"].get("gender") for _, v in arr], use_strong_sanitize=False)
    profiles = join_values([v["person"].get("profile_path") for _, v in arr], use_strong_sanitize=False)

    return names, ids, roles, genders, profiles


def extract_top_cast(cast):
    """
    주연 배우 Top 5 추출
    - cast를 order 기준 오름차순 정렬
    - character가 있는 항목만 사용
    - 상위 5명까지
    - 이름은 경미 정제, character는 강력 정제
    """
    if not cast:
        return "", "", "", "", "", ""

    sorted_cast = sorted(cast, key=lambda x: x.get("order", 999))
    filtered = [c for c in sorted_cast if c.get("character")]

    top_5 = filtered[:5]
    if not top_5:
        return "", "", "", "", "", ""

    names      = join_values([c.get("name") for c in top_5], use_strong_sanitize=False)
    ids        = join_values([c.get("id") for c in top_5], use_strong_sanitize=False)
    orders     = join_values([c.get("order") for c in top_5], use_strong_sanitize=False)
    characters = join_values([c.get("character") for c in top_5], use_strong_sanitize=True)
    genders    = join_values([c.get("gender") for c in top_5], use_strong_sanitize=False)
    profiles   = join_values([c.get("profile_path") for c in top_5], use_strong_sanitize=False)

    return names, ids, orders, characters, genders, profiles

# ==========================================================
# MOVIE DETAILS FETCHER
# ==========================================================
async def fetch_movie_detail(session, movie_id: int):
    url = f"{BASE_URL}/movie/{movie_id}"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "append_to_response": "credits",
    }

    data = await fetch_json(session, url, params, desc=f"movie/{movie_id}")
    if not data:
        return None

    imdb_id = data.get("imdb_id")
    if not imdb_id:
        return None

    vote_count = data.get("vote_count", 0)
    if vote_count < 30:
        return None

    title = sanitize_text_mild(data.get("title"))

    credits = data.get("credits", {})
    cast = credits.get("cast", []) or []
    crew = credits.get("crew", []) or []

    directors, director_ids, director_gender, director_profile = extract_directors(crew)
    writers, writer_ids, writer_roles, writer_gender, writer_profile = extract_writers(crew)
    top_cast, top_cast_ids, top_cast_order, characters, top_cast_gender, top_cast_profile = extract_top_cast(cast)

    return {
        "imdb_id": imdb_id,
        "movie_id": movie_id,
        "vote_count": vote_count,
        "title": title,

        "director_ids": director_ids,
        "directors": directors,
        "director_gender": director_gender,
        "director_profile_path": director_profile,

        "writer_ids": writer_ids,
        "writers": writers,
        "writer_roles": writer_roles,
        "writer_gender": writer_gender,
        "writer_profile_path": writer_profile,

        "top_cast_ids": top_cast_ids,
        "top_cast_order": top_cast_order,
        "top_cast": top_cast,
        "character": characters,
        "top_cast_gender": top_cast_gender,
        "top_cast_profile_path": top_cast_profile,
    }

# ==========================================================
# CHECKPOINT
# ==========================================================
def load_checkpoint():
    if CHECKPOINT_PATH.exists():
        try:
            data = json.loads(CHECKPOINT_PATH.read_text())
            return {
                "done_ids": set(data.get("done_ids", [])),
                "failed_ids": set(data.get("failed_ids", [])),
                "last_index": data.get("last_index", 0),
            }
        except Exception:
            return {"done_ids": set(), "failed_ids": set(), "last_index": 0}
    return {"done_ids": set(), "failed_ids": set(), "last_index": 0}


async def save_checkpoint(done_ids, failed_ids, last_index: int):
    async with checkpoint_lock:
        try:
            data = {
                "done_ids": list(done_ids),
                "failed_ids": list(failed_ids),
                "last_index": last_index,
                "timestamp": datetime.now().isoformat(),
            }
            tmp_path = CHECKPOINT_PATH.with_suffix(".tmp")
            tmp_path.write_text(json.dumps(data, ensure_ascii=False, indent=2))
            tmp_path.replace(CHECKPOINT_PATH)
        except Exception as e:
            print(f"⚠️ 체크포인트 저장 실패: {e}")


def load_done_from_jsonl():
    if not DETAILS_JSONL_PATH.exists():
        return set()

    done = set()
    with open(DETAILS_JSONL_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rec = json.loads(line)
                mid = rec.get("movie_id")
                if mid:
                    done.add(mid)
            except Exception:
                pass
    return done


def save_failed_ids(failed_ids):
    FAILED_IDS_PATH.write_text(
        json.dumps(sorted(list(failed_ids)), ensure_ascii=False, indent=2)
    )

# ==========================================================
# BATCH PROCESSING
# ==========================================================
async def process_batch(session, batch, done_ids: set, failed_ids: set):
    sem = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    async def worker(mid: int):
        if mid in done_ids or mid in failed_ids:
            return None
        async with sem:
            result = await fetch_movie_detail(session, mid)
            if result is None:
                failed_ids.add(mid)
                return None
            return result

    tasks = [asyncio.create_task(worker(mid)) for mid in batch]
    results = await asyncio.gather(*tasks)

    valid = [r for r in results if r is not None]

    if valid:
        async with jsonl_lock:
            try:
                lines = [json.dumps(r, ensure_ascii=False) + "\n" for r in valid]
                with open(DETAILS_JSONL_PATH, "a", encoding="utf-8") as f:
                    f.writelines(lines)

                for r in valid:
                    done_ids.add(r["movie_id"])

            except Exception as e:
                print(f"⚠️ JSONL 쓰기 실패: {e}")

    return len(valid)


def chunked(arr, n):
    for i in range(0, len(arr), n):
        yield arr[i:i+n]

# ==========================================================
# MAIN
# ==========================================================
async def main():
    start = datetime.now()

    async with aiohttp.ClientSession(headers=HEADERS) as session:
        if MOVIE_IDS_PATH.exists():
            movie_ids = json.loads(MOVIE_IDS_PATH.read_text())
            print(f"📂 기존 movie_ids 로드: {len(movie_ids):,}")
        else:
            movie_ids = await discover_movie_ids(session)

        checkpoint = load_checkpoint()
        done_ids   = checkpoint["done_ids"]
        failed_ids = checkpoint["failed_ids"]

        done_ids |= load_done_from_jsonl()

        remaining = [mid for mid in movie_ids if mid not in done_ids and mid not in failed_ids]
        print(f"⏳ 처리 대상: {len(remaining):,}")
        print(f"   완료: {len(done_ids):,}, 실패: {len(failed_ids):,}")

        if not DETAILS_JSONL_PATH.exists():
            DETAILS_JSONL_PATH.touch()

        BATCH_SIZE = 400
        total_processed = 0

        batches = list(chunked(remaining, BATCH_SIZE))

        for idx, batch in enumerate(batches, start=1):
            batch_start = datetime.now()
            n = await process_batch(session, batch, done_ids, failed_ids)
            total_processed += n

            elapsed = (datetime.now() - batch_start).total_seconds()
            print(f"  ▷ Batch {idx}/{len(batches)}: {n}개 처리, 누적={total_processed}, {elapsed:.1f}s")

            if idx % 10 == 0:
                await save_checkpoint(done_ids, failed_ids, idx)
                save_failed_ids(failed_ids)
                print(f"💾 체크포인트 저장 (batch={idx})")

        await save_checkpoint(done_ids, failed_ids, len(batches))
        save_failed_ids(failed_ids)

    # JSONL → Parquet
    print("\n📦 JSONL → Parquet 변환 중...")

    if not DETAILS_JSONL_PATH.exists():
        print("⚠️ JSONL 파일이 없어 Parquet 변환을 건너뜀")
        return

    df = pd.read_json(DETAILS_JSONL_PATH, lines=True)

    df = df[df["imdb_id"].notna()]
    df = df[df["movie_id"].notna()]
    df = df.drop_duplicates(subset=["movie_id"], keep="first").reset_index(drop=True)

    cols = [
        "imdb_id", "movie_id", "vote_count", "title",
        "director_ids", "directors", "director_gender", "director_profile_path",
        "writer_ids", "writers", "writer_roles", "writer_gender", "writer_profile_path",
        "top_cast_ids", "top_cast_order", "top_cast", "character",
        "top_cast_gender", "top_cast_profile_path",
    ]

    missing = [c for c in cols if c not in df.columns]
    if missing:
        print(f"⚠️ 누락 컬럼 자동 생성: {missing}")
        for c in missing:
            df[c] = ""

    df = df[cols]

    df["movie_id"]   = df["movie_id"].astype("int32")
    df["vote_count"] = df["vote_count"].astype("int32")

    for col in df.columns:
        if col not in ["movie_id", "vote_count"]:
            df[col] = df[col].astype("string")

    df.to_parquet(FINAL_PARQUET_PATH, index=False)

    elapsed = (datetime.now() - start).total_seconds() / 60
    print(f"\n🎉 전체 파이프라인 완료! ({elapsed:.1f}분)")
    print(f"📁 저장 파일: {FINAL_PARQUET_PATH}")
    print(f"   총 영화 수: {len(df):,}편")
    print(f"   실패 영화 수: {len(json.loads(FAILED_IDS_PATH.read_text())) if FAILED_IDS_PATH.exists() else 0:,}편")


if __name__ == "__main__":
    asyncio.run(main())

📌 Discover 단계 시작 (movie_id 수집)
📅 분할된 날짜 구간: 4
  ▷ [1/4] 2005-01-01 ~ 2010-03-25 (pages=266)
  ▷ [2/4] 2010-03-26 ~ 2015-06-17 (pages=364)
  ▷ [3/4] 2015-06-18 ~ 2020-09-08 (pages=460)
  ▷ [4/4] 2020-09-09 ~ 2025-11-30 (pages=331)
🎉 Discover 완료: 고유 movie_id 28,384개 확보
⏳ 처리 대상: 23,984
   완료: 4,399, 실패: 1
  ▷ Batch 1/60: 400개 처리, 누적=400, 11.4s
  ▷ Batch 2/60: 399개 처리, 누적=799, 11.4s
  ▷ Batch 3/60: 400개 처리, 누적=1199, 12.1s
  ▷ Batch 4/60: 400개 처리, 누적=1599, 11.8s
  ▷ Batch 5/60: 400개 처리, 누적=1999, 12.1s
  ▷ Batch 6/60: 400개 처리, 누적=2399, 12.0s
  ▷ Batch 7/60: 399개 처리, 누적=2798, 12.0s
  ▷ Batch 8/60: 399개 처리, 누적=3197, 12.0s
  ▷ Batch 9/60: 398개 처리, 누적=3595, 12.0s
  ▷ Batch 10/60: 398개 처리, 누적=3993, 12.1s
💾 체크포인트 저장 (batch=10)
  ▷ Batch 11/60: 400개 처리, 누적=4393, 12.1s
  ▷ Batch 12/60: 400개 처리, 누적=4793, 11.9s
  ▷ Batch 13/60: 400개 처리, 누적=5193, 11.9s
  ▷ Batch 14/60: 400개 처리, 누적=5593, 11.9s
  ▷ Batch 15/60: 400개 처리, 누적=5993, 11.8s
  ▷ Batch 16/60: 399개 처리, 누적=6392, 11.8s
  ▷ Batch 17/60: 400개 처리, 누적=

## 영화 크레딧 수집 코드 (인표, 파싱 후)

In [ ]:
HEADERS = {"accept": "application/json"}

def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

# 기본 준비
## 환경변수에서 API_KEY 읽어오기
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
session = create_session()

WRITER_JOBS = {
    "Writer", "Screenplay", "Story", "Creator",
    "Novel", "Comic Book", "Adaptation", "Screenstory"
}

def get_credit(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
    }

    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(f"Error fetching details for {movie_id}: {e}")
        return None

    # Cast 데이터 처리 (안전한 방식)
    cast = data.get("cast", [])
    if cast:
        cast_df = pd.DataFrame(cast)
        # 'order' 컬럼이 있는지 확인
        if 'order' in cast_df.columns:
            cast_df = cast_df.sort_values(by=["order"])
        cast_df = cast_df.head(5)

        # 필요한 컬럼만 선택 (존재하는 컬럼만)
        available_cols = ['id', 'gender', 'name', 'popularity', 'profile_path', 'character']
        existing_cols = [col for col in available_cols if col in cast_df.columns]
        cast_df = cast_df[existing_cols].rename(columns={'id': 'cast_id'})
        cast_df['movie_id'] = movie_id
    else:
        # 빈 DataFrame 생성 (컬럼 구조 유지)
        cast_df = pd.DataFrame(columns=['cast_id', 'gender', 'name', 'popularity',
                                        'profile_path', 'character', 'movie_id'])

    # Crew 데이터 처리 (안전한 방식)
    crew = data.get("crew", [])
    if crew:
        crew_df = pd.DataFrame(crew)
        available_cols = ['id', 'gender', 'name', 'popularity', 'profile_path', 'department', 'job']
        existing_cols = [col for col in available_cols if col in crew_df.columns]
        crew_df = crew_df[existing_cols].rename(columns={'id': 'crew_member_id'})
        crew_df['movie_id'] = movie_id

        # Writer와 Director 필터링
        if 'job' in crew_df.columns:
            writer_df = crew_df[crew_df['job'].isin(WRITER_JOBS)]
            director_df = crew_df[crew_df['job'] == 'Director']
        else:
            writer_df = pd.DataFrame(columns=crew_df.columns)
            director_df = pd.DataFrame(columns=crew_df.columns)
    else:
        # 빈 DataFrame 생성
        empty_cols = ['crew_member_id', 'gender', 'name', 'popularity',
                     'profile_path', 'department', 'job', 'movie_id']
        writer_df = pd.DataFrame(columns=empty_cols)
        director_df = pd.DataFrame(columns=empty_cols)

    return {"cast": cast_df, "writer": writer_df, "director": director_df}

if __name__ == "__main__":
    movies = pd.read_parquet('files/movies_filtered_f.parquet')
    ids = movies['id'].tolist()  # Series를 list로 변환하면 메모리 효율적

    all_cast_dfs = []
    all_writer_dfs = []
    all_director_dfs = []

    # 3. 상세 정보 수집 단계
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(get_credit, movie_id): movie_id for movie_id in ids}

        for future in tqdm(as_completed(futures), total=len(futures),
                          desc="영화 크레딧 수집", unit="movie"):
            try:
                result = future.result()
                if result:
                    all_cast_dfs.append(result['cast'])
                    all_writer_dfs.append(result['writer'])
                    all_director_dfs.append(result['director'])
            except Exception as e:
                movie_id = futures[future]
                print(f"영화 ID {movie_id} 처리 중 오류: {e}")

    def safe_concat(df_list, name):
        """빈 데이터를 안전하게 처리하면서 DataFrame을 병합"""
        if df_list and not all(df.empty for df in df_list):
            return pd.concat(df_list, ignore_index=True)

        print(f"경고: {name} 데이터가 비어 있거나 수집되지 않았습니다.")

        # 컬럼 구조 정의
        column_schemas = {
            'Cast': ['cast_id', 'gender', 'name', 'popularity', 'profile_path', 'character', 'movie_id'],
            'Writer': ['crew_member_id', 'gender', 'name', 'popularity', 'profile_path', 'department', 'job', 'movie_id'],
            'Director': ['crew_member_id', 'gender', 'name', 'popularity', 'profile_path', 'department', 'job', 'movie_id']
        }

        return pd.DataFrame(columns=column_schemas.get(name, []))

    # 결과 저장
    final_cast_df = safe_concat(all_cast_dfs, 'Cast')
    final_cast_df.to_parquet('files/movies_cast.parquet', index=False)
    print(f"Cast 데이터 저장 완료: {len(final_cast_df)} 행")

    final_writer_df = safe_concat(all_writer_dfs, 'Writer')
    final_writer_df.to_parquet('files/movies_writer.parquet', index=False)
    print(f"Writer 데이터 저장 완료: {len(final_writer_df)} 행")

    final_director_df = safe_concat(all_director_dfs, 'Director')
    final_director_df.to_parquet('files/movies_director.parquet', index=False)
    print(f"Director 데이터 저장 완료: {len(final_director_df)} 행")

## 드라마 크레딧 수집 코드 (예원, 파싱 전)

In [ ]:
"""
TMDB TV Series Data Collector - Async Version with Environment Variables
환경 변수를 사용하여 API 키를 안전하게 관리하는 개선된 버전
"""

import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import json
import time
from typing import List, Dict, Optional
import logging
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('tmdb_collector.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


class TMDBAsyncCollector:
    def __init__(self, api_key: str = None, 
                 start_date: str = None, 
                 end_date: str = None,
                 min_vote_count: int = None,
                 max_workers: int = None):
        # 환경 변수 또는 파라미터에서 설정 로드
        self.api_key = api_key or os.getenv('TMDB_API_KEY')
        if not self.api_key:
            raise ValueError("TMDB API key not found. Set TMDB_API_KEY in .env file or pass as parameter")
        
        self.start_date = start_date or os.getenv('START_DATE', '2005-01-01')
        self.end_date = end_date or os.getenv('END_DATE', '2025-11-30')
        self.min_vote_count = min_vote_count or int(os.getenv('MIN_VOTE_COUNT', '30'))
        
        self.base_url = "https://api.themoviedb.org/3"
        self.headers = {"accept": "application/json"}
        
        # Rate limiting
        self.rate_limit = max_workers or int(os.getenv('MAX_WORKERS', '40'))
        self.rate_period = 1.0
        self.semaphore = asyncio.Semaphore(self.rate_limit)
        
        # 파일 경로
        self.checkpoint_file = "checkpoint_async.json"
        self.output_file = "tmdb_tv_series_data.parquet"
        
        # 데이터 저장
        self.collected_data = []
        
        # 통계
        self.stats = {
            'total_requests': 0,
            'successful_requests': 0,
            'failed_requests': 0,
            'series_with_imdb': 0,
            'series_without_imdb': 0,
            'start_time': None,
            'end_time': None
        }
        
    async def _rate_limited_request(self, session: aiohttp.ClientSession, url: str, params: dict) -> Optional[dict]:
        """Rate limiting을 적용한 API 요청"""
        async with self.semaphore:
            self.stats['total_requests'] += 1
            
            try:
                async with session.get(url, params=params, headers=self.headers, timeout=aiohttp.ClientTimeout(total=30)) as response:
                    if response.status == 200:
                        self.stats['successful_requests'] += 1
                        return await response.json()
                    elif response.status == 429:  # Too Many Requests
                        retry_after = int(response.headers.get('Retry-After', 1))
                        logger.warning(f"Rate limited. Waiting {retry_after}s")
                        await asyncio.sleep(retry_after)
                        return await self._rate_limited_request(session, url, params)
                    else:
                        self.stats['failed_requests'] += 1
                        logger.error(f"Error {response.status}: {url}")
                        return None
            except asyncio.TimeoutError:
                self.stats['failed_requests'] += 1
                logger.error(f"Timeout: {url}")
                return None
            except Exception as e:
                self.stats['failed_requests'] += 1
                logger.error(f"Request failed: {e}")
                return None
            finally:
                await asyncio.sleep(1.0 / self.rate_limit)
    
    async def fetch_discover_page(self, session: aiohttp.ClientSession, 
                                  start_date: str, end_date: str, page: int) -> Optional[dict]:
        """Discover API로 TV 시리즈 목록 가져오기"""
        url = f"{self.base_url}/discover/tv"
        params = {
            "api_key": self.api_key,
            "first_air_date.gte": start_date,
            "first_air_date.lte": end_date,
            "vote_count.gte": self.min_vote_count,
            "include_adult": "false",
            "include_null_first_air_dates": "false",
            "language": "en-US",
            "page": page,
            "sort_by": "first_air_date.asc"
        }
        return await self._rate_limited_request(session, url, params)
    
    async def fetch_external_ids(self, session: aiohttp.ClientSession, series_id: int) -> Optional[dict]:
        """External IDs (IMDB ID 포함) 가져오기"""
        url = f"{self.base_url}/tv/{series_id}/external_ids"
        params = {"api_key": self.api_key}
        return await self._rate_limited_request(session, url, params)
    
    async def fetch_credits(self, session: aiohttp.ClientSession, series_id: int) -> Optional[dict]:
        """크레딧 정보 (배우, 제작진) 가져오기"""
        url = f"{self.base_url}/tv/{series_id}/credits"
        params = {
            "api_key": self.api_key,
            "language": "en-US"
        }
        return await self._rate_limited_request(session, url, params)
    
    def extract_executive_producers(self, crew_list: List[dict]) -> Dict[str, str]:
        """총괄 프로듀서 정보 추출"""
        producers = [c for c in crew_list if c.get("job") == "Executive Producer"]
        
        if not producers:
            return {
                "executive_producer_name": "",
                "executive_producer_ids": "",
                "executive_producer_gender": "",
                "executive_producer_profile_path": ""
            }
        
        return {
            "executive_producer_name": "; ".join([p.get("name", "") for p in producers]),
            "executive_producer_ids": "; ".join([str(p.get("id", "")) for p in producers]),
            "executive_producer_gender": "; ".join([str(p.get("gender", "")) for p in producers]),
            "executive_producer_profile_path": "; ".join([p.get("profile_path", "") or "" for p in producers])
        }
    
    def extract_writers(self, crew_list: List[dict]) -> Dict[str, str]:
        """작가 정보 추출"""
        writer_jobs = {"Writer", "Screenplay", "Story", "Creator", "Novel", "Comic Book"}
        writers = [
            c for c in crew_list
            if c.get("job") in writer_jobs or c.get("department") == "Writing"
        ]
        
        if not writers:
            return {
                "writers_name": "",
                "writer_roles": "",
                "writer_ids": "",
                "writer_gender": "",
                "writer_profile_path": ""
            }
        
        return {
            "writers_name": "; ".join([w.get("name", "") for w in writers]),
            "writer_roles": "; ".join([w.get("job", "") for w in writers]),
            "writer_ids": "; ".join([str(w.get("id", "")) for w in writers]),
            "writer_gender": "; ".join([str(w.get("gender", "")) for w in writers]),
            "writer_profile_path": "; ".join([w.get("profile_path", "") or "" for w in writers])
        }
    
    def extract_top_cast(self, cast_list: List[dict]) -> Dict[str, str]:
        """주연 배우 상위 5명 정보 추출 (order 기준)"""
        sorted_cast = sorted(cast_list, key=lambda x: x.get("order", 999))
        top_cast = [c for c in sorted_cast if c.get("character")][:5]
        
        if not top_cast:
            return {
                "top_cast_order": "",
                "top_cast": "",
                "character": "",
                "top_cast_ids": "",
                "top_cast_gender": "",
                "top_cast_profile_path": ""
            }
        
        return {
            "top_cast_order": "; ".join([str(c.get("order", "")) for c in top_cast]),
            "top_cast": "; ".join([c.get("name", "") for c in top_cast]),
            "character": "; ".join([c.get("character", "") for c in top_cast]),
            "top_cast_ids": "; ".join([str(c.get("id", "")) for c in top_cast]),
            "top_cast_gender": "; ".join([str(c.get("gender", "")) for c in top_cast]),
            "top_cast_profile_path": "; ".join([c.get("profile_path", "") or "" for c in top_cast])
        }
    
    async def process_series(self, session: aiohttp.ClientSession, series_data: dict) -> Optional[dict]:
        """개별 TV 시리즈 처리"""
        series_id = series_data.get("id")
        
        # External IDs 가져오기
        external_ids = await self.fetch_external_ids(session, series_id)
        if not external_ids or not external_ids.get("imdb_id"):
            self.stats['series_without_imdb'] += 1
            logger.debug(f"Series {series_id} has no IMDB ID, skipping")
            return None
        
        self.stats['series_with_imdb'] += 1
        
        # Credits 가져오기
        credits = await self.fetch_credits(session, series_id)
        if not credits:
            logger.debug(f"Failed to fetch credits for series {series_id}")
            return None
        
        # 데이터 추출
        cast_list = credits.get("cast", [])
        crew_list = credits.get("crew", [])
        
        row_data = {
            "imdb_id": external_ids.get("imdb_id"),
            "series_id": series_id,
            "title": series_data.get("name", ""),
            "original_name": series_data.get("original_name", ""),
            "first_air_date": series_data.get("first_air_date", ""),
            "vote_average": series_data.get("vote_average", 0),
            "vote_count": series_data.get("vote_count", 0),
            "popularity": series_data.get("popularity", 0),
        }
        
        # 총괄 프로듀서, 작가, 주연 배우 정보 추가
        row_data.update(self.extract_executive_producers(crew_list))
        row_data.update(self.extract_writers(crew_list))
        row_data.update(self.extract_top_cast(cast_list))
        
        return row_data
    
    async def collect_date_range(self, session: aiohttp.ClientSession, 
                                start_date: str, end_date: str) -> None:
        """특정 날짜 범위의 데이터 수집"""
        logger.info(f"Collecting: {start_date} to {end_date}")
        page = 1
        
        while page <= 500:  # TMDB 500페이지 제한
            result = await self.fetch_discover_page(session, start_date, end_date, page)
            
            if not result or "results" not in result:
                break
            
            results = result["results"]
            if not results:
                break
            
            total_pages = min(result.get("total_pages", 0), 500)
            logger.info(f"Processing page {page}/{total_pages} ({start_date} ~ {end_date})")
            
            # 각 시리즈 처리
            tasks = [self.process_series(session, series) for series in results]
            processed_data = await asyncio.gather(*tasks)
            
            # 유효한 데이터만 추가
            valid_data = [d for d in processed_data if d is not None]
            self.collected_data.extend(valid_data)
            
            logger.info(f"Collected {len(valid_data)} valid series from this page (Total: {len(self.collected_data)})")
            
            if page >= total_pages:
                break
            
            page += 1
        
        logger.info(f"Completed range {start_date} ~ {end_date}")
    
    def generate_date_ranges(self, start_date: str, end_date: str, months: int = 3) -> List[tuple]:
        """날짜 범위를 여러 구간으로 분할 (500페이지 제한 대응)"""
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        
        ranges = []
        current = start
        
        while current < end:
            next_date = current + timedelta(days=months * 30)
            if next_date > end:
                next_date = end
            
            ranges.append((
                current.strftime("%Y-%m-%d"),
                next_date.strftime("%Y-%m-%d")
            ))
            current = next_date + timedelta(days=1)
        
        return ranges
    
    def save_checkpoint(self, completed_ranges: List[tuple]) -> None:
        """진행 상황 저장"""
        checkpoint = {
            "completed_ranges": completed_ranges,
            "collected_count": len(self.collected_data),
            "timestamp": datetime.now().isoformat(),
            "stats": self.stats
        }
        with open(self.checkpoint_file, "w") as f:
            json.dump(checkpoint, f, indent=2)
        logger.info(f"Checkpoint saved: {len(completed_ranges)} ranges completed")
    
    def load_checkpoint(self) -> List[tuple]:
        """저장된 진행 상황 불러오기"""
        if not Path(self.checkpoint_file).exists():
            return []
        
        with open(self.checkpoint_file, "r") as f:
            checkpoint = json.load(f)
            logger.info(f"Loaded checkpoint: {checkpoint['collected_count']} series previously collected")
            return [tuple(r) for r in checkpoint.get("completed_ranges", [])]
    
    def save_to_parquet(self) -> None:
        """수집한 데이터를 Parquet 파일로 저장"""
        if not self.collected_data:
            logger.warning("No data to save")
            return
        
        df = pd.DataFrame(self.collected_data)
        df.to_parquet(self.output_file, index=False, engine='pyarrow')
        logger.info(f"Data saved to {self.output_file}: {len(df)} series")
    
    def print_statistics(self) -> None:
        """수집 통계 출력"""
        logger.info("\n" + "="*60)
        logger.info("Collection Statistics")
        logger.info("="*60)
        logger.info(f"Total Requests: {self.stats['total_requests']}")
        logger.info(f"Successful Requests: {self.stats['successful_requests']}")
        logger.info(f"Failed Requests: {self.stats['failed_requests']}")
        logger.info(f"Series with IMDB ID: {self.stats['series_with_imdb']}")
        logger.info(f"Series without IMDB ID: {self.stats['series_without_imdb']}")
        logger.info(f"Final Collected Series: {len(self.collected_data)}")
        
        if self.stats['start_time'] and self.stats['end_time']:
            duration = self.stats['end_time'] - self.stats['start_time']
            logger.info(f"Total Duration: {duration:.2f} seconds ({duration/60:.2f} minutes)")
            if self.stats['total_requests'] > 0:
                logger.info(f"Average Request Rate: {self.stats['total_requests']/duration:.2f} req/s")
        
        logger.info("="*60)
    
    async def run(self) -> None:
        """메인 수집 프로세스"""
        self.stats['start_time'] = time.time()
        
        logger.info("="*60)
        logger.info("TMDB TV Series Data Collector - Async Version")
        logger.info("="*60)
        logger.info(f"Period: {self.start_date} ~ {self.end_date}")
        logger.info(f"Min Vote Count: {self.min_vote_count}")
        logger.info(f"Rate Limit: {self.rate_limit} req/s")
        logger.info("="*60)
        
        # 날짜 범위 생성
        all_ranges = self.generate_date_ranges(self.start_date, self.end_date, months=3)
        completed_ranges = self.load_checkpoint()
        
        # 아직 수집하지 않은 범위 필터링
        remaining_ranges = [r for r in all_ranges if r not in completed_ranges]
        logger.info(f"Total ranges: {len(all_ranges)}, Remaining: {len(remaining_ranges)}")
        
        if not remaining_ranges:
            logger.info("All ranges already collected!")
            self.stats['end_time'] = time.time()
            self.print_statistics()
            return
        
        async with aiohttp.ClientSession() as session:
            for i, (range_start, range_end) in enumerate(remaining_ranges, 1):
                logger.info(f"\n{'='*60}")
                logger.info(f"Range {i}/{len(remaining_ranges)}")
                
                await self.collect_date_range(session, range_start, range_end)
                
                # 체크포인트 저장
                completed_ranges.append((range_start, range_end))
                self.save_checkpoint(completed_ranges)
                
                # 중간 저장 (매 5개 범위마다)
                if i % 5 == 0:
                    self.save_to_parquet()
        
        # 최종 저장
        self.save_to_parquet()
        
        self.stats['end_time'] = time.time()
        self.print_statistics()


async def main():
    try:
        collector = TMDBAsyncCollector()
        await collector.run()
    except ValueError as e:
        logger.error(f"Configuration error: {e}")
        logger.error("Please create a .env file with your TMDB_API_KEY")
    except Exception as e:
        logger.error(f"Unexpected error: {e}", exc_info=True)


if __name__ == "__main__":
    asyncio.run(main())